# Notebook explicatif du code Scraping, Cleaning et Analyse

# Scraping d'annonces immobilières sur ParuVendu


## Objectif du notebook

Ce notebook a pour objectif d'expliquer la partie scraping des annonces immobilières en vente sur le site **ParuVendu**, pour plusieurs villes françaises.

Les données récupérées sont :
- le titre de l’annonce
- le lien
- la description
- le prix
- la localisation
- les détails du bien (surface, pièces, etc.)

Les résultats sont stockés dans un fichier CSV, avec un système de reprise automatique en cas d’arrêt du script (checkpoint).


## Choix du site ParuVendu

Le site ParuVendu a été retenu pour plusieurs raisons :

- il propose un grand nombre d’annonces immobilières
- les annonces sont accessibles sans authentification
- la structure des pages est relativement stable
- les informations clés sont présentes dès la page de liste


## Choix des villes françaises

Le scraping porte sur une sélection de **20 villes françaises**.

Ce choix vise à obtenir :
- une couverture géographique nationale
- une diversité de marchés immobiliers
- un équilibre entre grandes métropoles et villes moyennes

Les villes sélectionnées sont :

- Paris
- Marseille
- Lyon
- Toulouse
- Nice
- Nantes
- Montpellier
- Strasbourg
- Bordeaux
- Lille
- Rennes
- Reims
- Toulon
- Saint-Étienne
- Le Havre
- Grenoble
- Dijon
- Angers
- Nîmes
- Clermont-Ferrand


## Import des bibliothèques

Cette section permet d’importer l’ensemble des bibliothèques nécessaires au fonctionnement du scraping.

Chaque bibliothèque a un rôle spécifique :
- `requests` : envoi de requêtes HTTP vers le site
- `BeautifulSoup` : analyse et navigation dans le code HTML
- `csv` : lecture et écriture des données dans un fichier CSV
- `time` : gestion des pauses entre les requêtes
- `os` : gestion des chemins et des fichiers
- `json` : sauvegarde et lecture du checkpoint

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import os
import re
import json

## Configuration générale du scraping

Cette section regroupe les paramètres globaux du script.

Dès cette étape, l’objectif est de limiter les risques de blocage en simulant le comportement d’un navigateur réel.

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "fr-FR,fr;q=0.9,en;q=0.8",
}

## Sélection des villes françaises

Le scraping est réalisé ville par ville afin de :
- structurer la collecte
- contrôler le volume de données
- limiter la charge envoyée au site

Les villes sélectionnées représentent différents marchés immobiliers.

In [ ]:
villes = [
    "paris-75", "marseille", "lyon", "toulouse", "nice",
    "nantes", "montpellier", "strasbourg", "bordeaux", "lille",
    "rennes", "reims", "toulon", "saint-etienne", "le-havre",
    "grenoble", "dijon", "angers", "nimes", "clermont-ferrand"
]

## Paramètres de contrôle et de sécurité

Ces paramètres permettent de :
- limiter le nombre de pages parcourues
- éviter une collecte trop agressive
- encadrer chaque exécution du script

In [ ]:
nb_pages = 5
url_base = "https://www.paruvendu.fr/immobilier/vente/"

MAX_ANNONCES_PAR_RUN = 1000
CHECKPOINT_FILE = "checkpoint.json"

## Gestion du fichier CSV

Un problème rapidement identifié est la perte de données en cas d’arrêt brutal du script (blocage, CAPTCHA, erreur réseau).

Pour éviter cela, le script recharge les annonces déjà collectées avant d’en ajouter de nouvelles.

In [ ]:
BASE_DIR = os.getcwd()
csv_file = os.path.join(BASE_DIR, "..", "DATA", "ANNONCES_RAW.csv")

existing_rows = []

if os.path.exists(csv_file):
    with open(csv_file, "r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            existing_rows.append(row)

scraped_rows = []

## Problème clé : récupération de la localisation

Les pages de résultats permettent de récupérer :
- titre
- prix
- description

En revanche, la localisation précise (quartier, arrondissement) n’est disponible que sur la page individuelle de chaque annonce.

## Mise en place du système de checkpoint

L’accès répété aux pages de détail déclenche un blocage du site.
Un simple ralentissement ne suffit pas.

La solution retenue consiste à fragmenter le scraping grâce à un système de checkpoint.

In [ ]:
def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        try:
            with open(CHECKPOINT_FILE, "r", encoding="utf-8") as f:
                data = json.load(f)
                return {
                    "ville_index": int(data.get("ville_index", 0)),
                    "page": int(data.get("page", 1)),
                }
        except Exception:
            pass
    return {"ville_index": 0, "page": 1}


def save_checkpoint(ville_index, page):
    data = {"ville_index": ville_index, "page": page}
    with open(CHECKPOINT_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f)


## Détection du blocage (CAPTCHA)

Lorsque le site détecte un trafic inhabituel, il affiche une page de protection.

Cette fonction permet d’identifier ce blocage et d’arrêter immédiatement le scraping.


In [ ]:
def is_captcha(html: str) -> bool:
    return (
        "Nos systèmes ont détecté un trafic inhabituel" in html
        or "Je ne suis pas un robot" in html
    )

## Création de la session HTTP

Une session permet de conserver les paramètres et d’améliorer la stabilité des requêtes.

In [ ]:
session = requests.Session()
session.headers.update(headers)

## Initialisation du scraping

Le script reprend automatiquement à partir du dernier checkpoint enregistré.

In [ ]:
checkpoint = load_checkpoint()
start_ville_index = checkpoint["ville_index"]
start_page = checkpoint["page"]

stop_scraping = False
nb_annonces_run = 0

## Boucle principale de scraping

Cette boucle constitue le cœur du script.

Elle parcourt :
1. les villes
2. les pages de résultats
3. les annonces individuelles

C’est à cette étape que les blocages apparaissent.

In [ ]:
for i_ville, ville in enumerate(villes):
    if stop_scraping:
        break

    if i_ville < start_ville_index:
        continue

    for page in range(1, nb_pages + 1):
        if stop_scraping:
            break

        if i_ville == start_ville_index and page < start_page:
            continue

        url_ville = f"{url_base}{ville}/?p={page}&allp=1"

        response = session.get(url_ville, timeout=10)

        if is_captcha(response.text):
            save_checkpoint(i_ville, page)
            stop_scraping = True
            break

        soup = BeautifulSoup(response.text, "lxml")
        annonces = soup.find_all("div", class_="blocAnnonce")

        for a in annonces:
            if nb_annonces_run >= MAX_ANNONCES_PAR_RUN:
                save_checkpoint(i_ville, page)
                stop_scraping = True
                break

            annonce_h3 = a.find("h3")
            if not annonce_h3:
                continue

            annonce_title = annonce_h3.find("a")
            if not annonce_title or not annonce_title.has_attr("href"):
                continue

            title = annonce_title.get("title", "").strip()
            lien = "https://www.paruvendu.fr" + annonce_title["href"]

            description_tag = a.find("p", class_="text-justify")
            description = description_tag.get_text(strip=True) if description_tag else ""

            price_tag = a.find("div", class_="encoded-lnk")
            price = price_tag.get_text(strip=True) if price_tag else ""

            detail = session.get(lien, timeout=10)

            if is_captcha(detail.text):
                save_checkpoint(i_ville, page)
                stop_scraping = True
                break

            soup_loc = BeautifulSoup(detail.text, "lxml")
            loc_tag = soup_loc.find("span", id="detail_loc")
            if not loc_tag:
                loc_tag = soup_loc.find("span", class_="ttldetail_loc1h")

            localisation = loc_tag.get_text(strip=True) if loc_tag else ""

            details = [d.get_text(strip=True) for d in a.select("div.flex.flex-wrap.gap-x-3 > *")]

            scraped_rows.append({
                "Ville": ville,
                "Titre": title,
                "Lien": lien,
                "Description": description,
                "Prix": price,
                "Localisation": localisation,
                "Détails": ", ".join(details),
            })

            nb_annonces_run += 1
            time.sleep(1)

        save_checkpoint(i_ville, page + 1)
        time.sleep(2)


## Fusion et sauvegarde finale

Les données sont fusionnées, les doublons supprimés, et le fichier CSV est mis à jour.

In [ ]:
all_rows = existing_rows + scraped_rows

seen = set()
unique_rows = []

for r in all_rows:
    lien = r.get("Lien")
    if lien and lien not in seen:
        unique_rows.append(r)
        seen.add(lien)

os.makedirs(os.path.dirname(csv_file), exist_ok=True)

with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["Ville", "Titre", "Lien", "Description", "Prix", "Localisation", "Détails"],
    )
    writer.writeheader()
    writer.writerows(unique_rows)

## Conclusion

Le développement de ce script montre que le scraping nécessite une adaptation constante aux contraintes du site.

Le principal obstacle a été le blocage lors de l’accès aux pages de détail nécessaires à la récupération de la localisation.

Le système de checkpoint constitue la réponse centrale à ce problème.

# ---------------------------------------------------------

# Explication CLEAN_DATA.py

# Nettoyage et structuration des données immobilières

Après la phase de scraping, les données collectées sont brutes et hétérogènes.

Ce notebook présente l’ensemble du processus de nettoyage, de standardisation et d’enrichissement des données, afin d’obtenir une base exploitable pour l’analyse statistique.


## Import des bibliothèques

Les bibliothèques utilisées permettent :
- la manipulation des données (pandas, numpy)
- le traitement de texte (re)
- la gestion des fichiers
- le géocodage optionnel des localisations

In [ ]:
import pandas as pd
import re
import os
import numpy as np
from geopy.geocoders import Nominatim
from time import sleep


## Chargement des fichiers CSV

Les données issues du scraping sont stockées dans un fichier CSV brut.
Un second fichier est généré après nettoyage.

Cette séparation permet de conserver une trace des données originales et d’éviter toute perte d’information.

In [ ]:
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
csv_file = os.path.join(BASE_DIR, "..", "DATA", "ANNONCES_RAW.csv")
clean_csv_file = os.path.join(BASE_DIR, "..", "DATA", "ANNONCES_CLEAN.csv")

print("Chemin CSV raw :", os.path.abspath(csv_file))
print("Fichier existe ?", os.path.exists(csv_file))

df = pd.read_csv(csv_file)

## Standardisation des noms de villes

Lors du scraping, certaines villes contiennent un suffixe correspondant au département (exemple : paris-75).

Ces variations empêchent une analyse cohérente.
Les noms de villes sont donc standardisés.

In [ ]:
df["Ville"] = (
    df["Ville"]
    .str.replace(r"-\d+$", "", regex=True)
    .str.replace("-", " ")
    .str.title()
)

## Nettoyage des champs texte

Les champs Description et Détails contiennent :
- des sauts de ligne
- des caractères spéciaux
- des espaces multiples

Ces éléments compliquent l’extraction d’informations
et doivent être supprimés.

In [ ]:
df["Détails"] = (
    df["Détails"]
    .astype(str)
    .str.replace("\n"," ")
    .str.replace("\xa0"," ")
)

df["Description"] = (
    df["Description"]
    .astype(str)
    .str.replace("\n"," ")
    .str.replace("_", " ")
    .str.replace(" -", " ")
    .str.replace(r"\s+", " ", regex=True)
)

## Problème des détails non structurés

Les informations importantes (nombre de pièces, chambres, garage, DPE) sont présentes dans une seule chaîne de texte non structurée.

Il est donc nécessaire de créer une fonction spécifique pour extraire ces informations de manière fiable.

In [ ]:
def parse_details(details_str):
    details_str = str(details_str)
    out = {
        "Pieces": None,
        "Chambres": None,
        "Garage": 0,
        "Balcon": 0,
        "Ascenseur": 0,
        "Terrain_m2": None,
        "DPE": None,
    }

    pieces_match = re.search(r"(\d+)\s*pi[eè]ce", details_str, re.I)
    if pieces_match:
        out["Pieces"] = int(pieces_match.group(1))

    chambres_match = re.search(r"(\d+)\s*chambre", details_str, re.I)
    if chambres_match:
        out["Chambres"] = int(chambres_match.group(1))

    for col in ["Garage", "Balcon", "Ascenseur"]:
        if col.lower() in details_str.lower():
            out[col] = 1

    terrain_match = re.search(r"terrain\s*(\d+)", details_str, re.I)
    if terrain_match:
        out["Terrain_m2"] = float(terrain_match.group(1))

    dpe_match = re.search(r"DPE\s*:\s*([A-G])", details_str, re.I)
    if dpe_match:
        out["DPE"] = dpe_match.group(1).upper()

    return out

## Problème du prix

Le champ Prix contient plusieurs informations :
- le prix total
- parfois le prix au mètre carré
- des caractères spéciaux

Ces informations doivent être séparées et converties en valeurs numériques.

In [ ]:
# Prix au m²
df["Prix_m2"] = (
    df["Prix"]
    .str.extract(r"([\d\s\u202f\xa0]+)\s*€\s*/\s*m2")[0]
    .str.replace(r"[\s\u202f\xa0]", "", regex=True)
    .astype(float)
)

In [ ]:
# Prix de vente
df["Prix"] = df["Prix"].str.replace(r"\*?\d[\d\s\u202f]+€ / m2", "", regex=True)
df["Prix"] = (
    df["Prix"]
    .str.strip()
    .str.replace("€", "", regex=False)
    .str.replace(" ", "", regex=False)
    .str.replace("*", "", regex=False)
    .replace("NC", np.nan)
    .astype(float)
)

df = df.rename(columns={"Prix": "Prix_de_vente"})


## Application du parsing

La fonction de parsing est appliquée à l’ensemble du DataFrame.
Chaque information devient une colonne distincte.

In [ ]:
details_df = df["Détails"].apply(parse_details).apply(pd.Series)

## Extraction de la surface

La surface du bien est souvent présente uniquement dans le titre.
Une extraction par expression régulière est donc nécessaire.

In [ ]:
def extract_surface(title):
    if pd.isna(title):
        return None
    match = re.search(r"(\d+)\s*m²", title.replace("\xa0",""))
    return int(match.group(1)) if match else None

df["Surface_m2"] = df["Titre"].apply(extract_surface)

## Identification du type de bien

Le type de bien est déduit du début du titre.
Certaines catégories sont regroupées pour homogénéiser l’analyse et se concentre sur Appartement ou Maison

In [ ]:
df["type"] = df["Titre"].str.split("-").str[0].str.strip()
df["type"] = df["type"].str.replace("loft", "Appartement", case=False)
df["type"] = df["type"].str.replace("villa", "Maison", case=False)
df["type"] = df["type"].str.replace("Duplex/triplex", "Appartement", case=False)


## Fusion des données

Les colonnes extraites sont fusionnées avec le DataFrame principal sans créer de doublons.

In [ ]:
df_final = pd.concat(
    [df.drop(columns=["Détails"], errors="ignore"), details_df],
    axis=1
)

## Typage des variables

Les variables numériques sont converties dans des formats adaptés afin de permettre les analyses statistiques.

In [ ]:
cols_int = ["Pieces", "Chambres", "Garage", "Balcon", "Ascenseur"]
cols_float = ["Terrain_m2", "Surface_m2", "Prix_m2", "Prix_de_vente"]

for col in cols_int:
    if col in df_final.columns:
        df_final[col] = pd.to_numeric(df_final[col], errors="coerce").astype("Int64")

for col in cols_float:
    if col in df_final.columns:
        df_final[col] = pd.to_numeric(df_final[col], errors="coerce")


## Exclusion des biens non résidentiels

Certains types de biens ne sont pas pertinents pour l’analyse du marché résidentiel et sont donc exclus.

In [ ]:
types_a_exclure = [
    "terrain", "hotel", "hôtel", "peniche", "péniche", "garage", "boutique",
    "commerce", "bureau", "immeuble", "chateau", "château", "grange",
    "hangar", "parking", "box", "ferme", "plateau",
    "Parking / Garage", "Hôtel Particulier", "Propriété/château"
]

df_final = df_final[
    ~df_final["type"].str.lower().isin([t.lower() for t in types_a_exclure])
]


## Géocodage des localisations

Le géocodage permet d’obtenir des coordonnées GPS à partir des localisations textuelles.

In [ ]:
RUN_GEOCODING = False

## Nettoyage des valeurs aberrantes

Les valeurs extrêmes peuvent biaiser l’analyse.
Une première filtration est effectuée, puis une méthode basée sur l’IQR est appliquée.

In [ ]:
df_final = df_final[
    (df_final["Prix_m2"] >= 500) & (df_final["Prix_m2"] <= 20000)
]

def remove_outliers_iqr(df_final, col):
    Q1 = df_final[col].quantile(0.25)
    Q3 = df_final[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df_final[(df_final[col] >= lower) & (df_final[col] <= upper)]

df_final = remove_outliers_iqr(df_final, "Prix_m2")
df_final = remove_outliers_iqr(df_final, "Prix_de_vente")

## Sauvegarde du jeu de données final

Le jeu de données propre est sauvegardé et prêt pour l’analyse statistique.

In [ ]:
df_final.to_csv(clean_csv_file, index=False, encoding="utf-8")
print("✅ ANNONCES_CLEAN.csv généré :", os.path.abspath(clean_csv_file))

# --------------------------------------------------

# EXPLICATION ANALYSE

# Analyse exploratoire du marché immobilier

Après les phases de scraping et de nettoyage, les données sont suffisamment structurées pour permettre une analyse exploratoire.

L’objectif de cette analyse est double :
- comprendre les grandes tendances du marché immobilier
- identifier les relations entre les caractéristiques des biens et leur prix

Pour faciliter l’exploration, une application interactive a été développée à l’aide de la bibliothèque Streamlit.

## Import des bibliothèques

Plusieurs bibliothèques sont utilisées :
Streamlit : interface interactive
Pandas : manipulation des données
Matplotlib & Seaborn : visualisations statistiques
NumPy : calculs numériques
PyDeck : cartographie (heatmap géographique)
theme : personnalisation graphique du projet

In [ ]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pydeck as pdk
from matplotlib.colors import LinearSegmentedColormap
from theme import load_css, load_matplotlib_theme, COLORS

## Configuration de l’application

La page est configurée en mode large et avec un thème personnalisé afin d’améliorer la lisibilité des graphiques et des indicateurs clés.


In [ ]:
st.set_page_config(page_title="Analyse Immobilier", layout="wide")
load_css()
load_matplotlib_theme()
st.title("📊 Analyse du marché immobilier")

## Chargement des données nettoyées

Les données utilisées proviennent du fichier nettoyé issu de la phase précédente.

Un système de cache est utilisé afin d’optimiser les performances de l’application.

Problème rencontré :
Les noms de colonnes contenaient des espaces et des majuscules,
ce qui compliquait leur manipulation.

Solution :
Standardisation systématique des noms de colonnes.

In [ ]:
@st.cache_data
def load_data():
    base_dir = os.path.dirname(os.path.abspath(__file__))
    csv_path = os.path.join(base_dir, "..", "..", "DATA", "ANNONCES_CLEAN.CSV")
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()
    return df

df = load_data()

## Filtres interactifs

L’analyse repose sur des filtres dynamiques permettant d’explorer différentes configurations du marché.

Tous les filtres sont stockés dans st.sessions_state avec un bouton permetant de les réinitialiser.

Problème rencontré :
Un problème fréquent avec Streamlit est la réinitialisation involontaire des filtres :
- lors d’un clic
- lors d’un changement d’onglet
- lors d’un rerun automatique

Solution mise en place :
Un système de state management manuel a été implémenté.

In [ ]:
def init_state(_df: pd.DataFrame):
    villes_all = sorted(_df["ville"].dropna().unique().tolist())
    pieces_all = sorted(_df["pieces"].dropna().unique().tolist())
    types_all = sorted(_df["type"].dropna().unique().tolist())
    dpe_all = sorted(_df["dpe"].dropna().unique().tolist())

    min_s = int(_df["surface_m2"].min())
    max_s = int(_df["surface_m2"].max())
    min_p = int(_df["prix_de_vente"].min())
    max_p = int(_df["prix_de_vente"].max())

    st.session_state["villes"] = villes_all
    st.session_state["surface"] = (min_s, max_s)
    st.session_state["prix"] = (min_p, max_p)
    st.session_state["pieces"] = pieces_all
    st.session_state["types"] = types_all
    st.session_state["dpe"] = dpe_all

    st.session_state["garage"] = False
    st.session_state["balcon"] = False
    st.session_state["ascenseur"] = False

    st.session_state["_reset_filters"] = False
    st.session_state["_inited"] = True

## Réinitialisation des filtres

L’accumulation de filtres pouvait conduire
à des sous-échantillons vides.

Un bouton de réinitialisation a été ajouté
pour améliorer l’expérience utilisateur.

In [ ]:
def request_reset():
    st.session_state["_reset_filters"] = True


## Logique d’initialisation globale

Cette cellule vérifie si l’état doit être initialisé ou réinitialisé.
Elle garantit un comportement stable lors des interactions utilisateur.

In [ ]:
if "_inited" not in st.session_state:
    st.session_state["_inited"] = False
if "_reset_filters" not in st.session_state:
    st.session_state["_reset_filters"] = False

if (not st.session_state["_inited"]) or st.session_state["_reset_filters"]:
    init_state(df)

min_s, max_s = int(df["surface_m2"].min()), int(df["surface_m2"].max())
min_p, max_p = int(df["prix_de_vente"].min()), int(df["prix_de_vente"].max())

## Interface des filtres latéraux
Les filtres permettent de segmenter le marché selon différents critères :
localisation
surface
prix
caractéristiques du bien

__Initialisation de l’interface de filtrage__

La sidebar commence par un titre explicite afin de séparer clairement l’espace
de contrôle de l’espace de visualisation.

Un bouton de réinitialisation globale est ajouté.
Il permet de remettre tous les filtres à leur état initial sans recharger
l’application, ce qui améliore fortement l’expérience utilisateur.

Ce mécanisme repose sur le session_state de Streamlit, utilisé comme mémoire
temporaire des paramètres sélectionnés.

In [ ]:
st.sidebar.header("Filtres")
st.sidebar.button("🔄 Réinitialiser les filtres", on_click=request_reset)

__Filtrage par localisation (villes)__
Un multiselect est utilisé afin de permettre :

La sélection d’une seule ville

La comparaison de plusieurs villes simultanément

L’absence de sélection, correspondant à une analyse nationale

In [ ]:
villes = st.sidebar.multiselect(
    "Ville",
    options=sorted(df["ville"].dropna().unique()),
    key="villes",
)

__Filtrage par surface__

Un slider double borne est utilisé pour explorer différents segments du marché (petites surfaces, grands logements)


Les bornes minimales et maximales sont calculées à partir du jeu de données,
ce qui empêche toute sélection hors domaine.

In [ ]:
surface_range = st.sidebar.slider(
    "Surface (m²)",
    min_value=min_s,
    max_value=max_s,
    value=(min_s, max_s),
    key="surface",
)


__Filtrage par prix__

Le filtre sur le prix fonctionne selon la même logique que celui sur la surface.

In [ ]:
prix_range = st.sidebar.slider(
    "Prix (€)",
    min_value=min_p,
    max_value=max_p,
    value=(min_p, max_p),
    key="prix",
)


__Décomposition des intervalles sélectionnés__

Cette étape améliore la lisibilité du code

In [ ]:
surface_min, surface_max = surface_range
prix_min, prix_max = prix_range

__Filtrage par nombre de pièces__

Le multiselect permet :

Une sélection flexible (ex. uniquement T2 et T3)

Une analyse comparative entre plusieurs typologies de logements

In [ ]:
pieces = st.sidebar.multiselect(
    "Nombre de pièces",
    options=sorted(df["pieces"].dropna().unique()),
    key="pieces",
)


__Filtrage par type de bien__

Permet de distinguer les deux grandes catégories du marché appartement et maison

In [ ]:
types_bien = st.sidebar.multiselect(
    "Type de bien",
    options=sorted(df["type"].dropna().unique()),
    key="types",
)


__Filtrage par performance énergétique DPE__

Ce filtre permet :

- D’analyser l’impact énergétique sur le prix au m²
- D’observer des tendances liées aux classes énergétiques
- D’isoler les biens les plus performants ou les plus énergivores

In [ ]:
dpe_selected = st.sidebar.multiselect(
    "DPE",
    options=sorted(df["dpe"].dropna().unique()),
    key="dpe",
)

__Filtres sur les équipements (variables binaires)__

Les checkboxes permettent une activation simple et intuitive :

- Inclusion uniquement des biens disposant de l’équipement
- Comparaison directe avec les biens standards
- Analyse de la valeur ajoutée de certains équipements

In [ ]:
garage_filter = st.sidebar.checkbox("Avec garage", key="garage")
balcon_filter = st.sidebar.checkbox("Avec balcon", key="balcon")
ascenseur_filter = st.sidebar.checkbox("Avec ascenseur", key="ascenseur")

## Application des filtres sur le jeu de données
Les filtres sont appliqués de manière progressive afin d’éviter toute exclusion involontaire.

In [ ]:
df_filtre = df.copy()

if villes:
    df_filtre = df_filtre[df_filtre["ville"].isin(villes)]

df_filtre = df_filtre[
    df_filtre["surface_m2"].between(surface_min, surface_max)
    & df_filtre["prix_de_vente"].between(prix_min, prix_max)
]

if pieces:
    df_filtre = df_filtre[df_filtre["pieces"].isin(pieces)]
if types_bien:
    df_filtre = df_filtre[df_filtre["type"].isin(types_bien)]
if dpe_selected:
    df_filtre = df_filtre[df_filtre["dpe"].isin(dpe_selected)]
if garage_filter:
    df_filtre = df_filtre[df_filtre["garage"].fillna(0) > 0]
if balcon_filter:
    df_filtre = df_filtre[df_filtre["balcon"].fillna(0) > 0]
if ascenseur_filter:
    df_filtre = df_filtre[df_filtre["ascenseur"].fillna(0) > 0]


## Structuration de l’analyse

Pour améliorer la lisibilité,
l’analyse est organisée en plusieurs onglets,
chacun correspondant à une dimension du marché.


In [ ]:
tab1, tab2, tab3, tab4, tab5, tab6 = st.tabs([
    "📊 Vue d’ensemble",
    "📐 Prix & Surfaces",
    "🏙️ Localisation",
    "🏠 Caractéristiques",
    "⚡ DPE",
    "🔗 Corrélations"
])


## Vue d’ensemble

Cette première analyse permet
d’observer la distribution globale
des prix et des surfaces.


In [ ]:
with tab1:
    col1, col2 = st.columns(2)

    with col1:
        fig, ax = plt.subplots()
        sns.histplot(
            df_filtre["prix_de_vente"],
            bins=30,
            kde=True,
            color=COLORS["prix"],
            ax=ax
        )
        ax.set_title("Distribution des prix de vente")
        st.pyplot(fig, use_container_width=True)
        plt.close(fig)

    with col2:
        fig, ax = plt.subplots()
        sns.histplot(
            df_filtre["surface_m2"],
            bins=30,
            kde=True,
            color=COLORS["surface"],
            ax=ax
        )
        ax.set_title("Distribution des surfaces")
        st.pyplot(fig, use_container_width=True)
        plt.close(fig)



## Analyse spatiale et carte de chaleur des prix au m²
La carte de chaleur met en évidence les zones de forte valorisation immobilière.

__Principe__ :

Chaque annonce est représentée par un point géographique (latitude, longitude).
La couche HeatmapLayer agrège ces points et met en évidence les zones où les prix au m² sont les plus élevés.

Cette approche permet :
- d’identifier des pôles de tension immobilière
- de comparer visuellement les grandes métropoles
- de dépasser une simple analyse par ville

__Préparation des données__ :

Seules les annonces disposant de coordonnées géographiques et d’un prix au m² sont conservées.

In [ ]:
df_map = df_filtre.dropna(subset=["latitude", "longitude", "prix_m2"])

Construction de la couche Heatmap

In [ ]:
layer = pdk.Layer(
    "HeatmapLayer",
    data=df_map,
    get_position='[longitude, latitude]',
    radiusPixels=70,
)

>radiusPixels contrôle la zone d’influence de chaque point 

>une valeur élevée permet de lisser la carte

>cela évite une lecture trop granulaire à l’échelle nationale

Définition du point de vue

In [ ]:
view_state = pdk.ViewState(
    latitude=float(df_map["latitude"].median()),
    longitude=float(df_map["longitude"].median()),
    zoom=4.5
)

Le point de vue est centré automatiquement sur la médiane des coordonnées, ce qui permet une adaptation dynamique aux filtres.

__Affichage de la carte__

In [ ]:
st.pydeck_chart(pdk.Deck(layers=[layer], initial_view_state=view_state))

## Relation entre surface et prix

Cette analyse permet de visualiser
la relation entre la taille du bien
et son prix de vente.


In [ ]:
with tab2:
    fig, ax = plt.subplots(figsize=(8, 3))
    sns.scatterplot(
        data=df_filtre,
        x="surface_m2",
        y="prix_de_vente",
        hue="type",
        palette="bright",
        alpha=0.6,
        s=25,
        ax=ax
    )
    sns.regplot(
        data=df_filtre,
        x="surface_m2",
        y="prix_de_vente",
        scatter=False,
        color="#ffd166",
        ax=ax
    )
    ax.set_xlabel("Surface (m²)")
    ax.set_ylabel("Prix (€)")
    st.pyplot(fig, use_container_width=True)
    plt.close(fig)


## Analyse par ville

Le prix au mètre carré est comparé entre les villes,
ce qui permet d’identifier les disparités géographiques.


In [ ]:
with tab3:
    order = df_filtre.groupby("ville")["prix_m2"].median().sort_values().index
    fig, ax = plt.subplots(figsize=(8,4))
    sns.barplot(
        data=df_filtre,
        x="ville",
        y="prix_m2",
        order=order,
        estimator=np.median,
        errorbar=None,
        ax=ax
    )
    plt.xticks(rotation=45)
    st.pyplot(fig)


## Impact des caractéristiques du bien

Cette analyse permet d’évaluer
la valorisation associée à certains équipements.


In [ ]:
with tab4:
    for opt in ["balcon", "garage", "ascenseur"]:
        fig, ax = plt.subplots()
        sns.boxplot(
            data=df_filtre,
            x=df_filtre[opt].map({0: "Non", 1: "Oui"}),
            y="prix_m2",
            ax=ax
        )
        st.pyplot(fig)


## Impact du DPE

Le Diagnostic de Performance Énergétique
constitue un critère de plus en plus structurant
sur le marché immobilier.


In [ ]:
with tab5:
    fig, ax = plt.subplots()
    sns.boxplot(
        data=df_filtre,
        x="dpe",
        y="prix_m2",
        order=["A","B","C","D","E","F","G"],
        ax=ax
    )
    st.pyplot(fig)


## Analyse des corrélations

La matrice de corrélation permet
d’identifier les relations linéaires
entre les principales variables quantitatives.


In [ ]:
with tab6:
    vars_corr = ["prix_de_vente","prix_m2","surface_m2","pieces","chambres"]
    fig, ax = plt.subplots(figsize=(6,5))
    sns.heatmap(
        df_filtre[vars_corr].corr(),
        annot=True,
        fmt=".2f",
        cmap="coolwarm",
        ax=ax
    )
    st.pyplot(fig)


## Conclusion de l’analyse exploratoire

L’analyse met en évidence :
- une forte relation entre surface et prix
- des écarts importants entre villes
- une valorisation des biens bien équipés
- un impact mesurable du DPE

Ces résultats constituent une base solide
pour la validation ou la réfutation
des hypothèses de recherche.
